In [3]:
import numpy as np
#import _pickle as pickle
#import glob
#from collections import OrderedDict
#from sklearn.preprocessing import OneHotEncoder, LabelEncoder
#from music21 import note, chord, instrument, converter
#import matplotlib.pyplot as plt
#%matplotlib inline 
from lib.preprocessing import preprocess_raw_data, load_piece
from lib.lstm_model import train

In [ ]:
piece = load_piece('./data/Mozart/', 2)
piece.shape

In [ ]:
train(piece, 1, 1, batch_size=20, model_save_name='./checkpoints/0/model')

In [30]:
from lib.preprocessing import load_piece
from lib.lstm_model import train
piece = load_piece('./data/Mozart/', 10)
print(piece.shape)
#train(piece, 1, 1, batch_size=20, model_save_name='./checkpoints/0/model')


(1939, 3, 19)


In [2]:
piece.shape


(7404, 3, 19)

In [6]:
def rearrange_data(data, num_comb=100):
    new_len = data.shape[0] - num_comb
    new_data = np.zeros([ new_len, 100, 3, 19])
    for i in range(new_len):
        new_data[i] = data[i:i+num_comb]
    return new_data

In [7]:
npiece = rearrange_data(piece)

In [31]:
piece[100,2]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1],
      dtype=int32)

In [ ]:
def prepare_data(notes, n_vocab):
    ''' create input sequences and output notes '''
    sequence_length = 100
    NetworkInput = []
    NetworkOutput = []
    # create a mapping to the notes
    mapper = LabelEncoder()
    mapped_notes = mapper.fit_transform(notes)

    for i in range(len(notes)-sequence_length):
        in_seq = mapped_notes[i : i+sequence_length]
        out_note = mapped_notes[i+sequence_length]
        NetworkInput.append(in_seq)
        NetworkOutput.append(out_note)
    
    n_patterns = len(NetworkOutput)
    
    NetworkInput = np.reshape(NetworkInput, (n_patterns, sequence_length, 1))
    NetworkInput = NetworkInput / float(n_vocab)
    
    NetworkOutput = np.reshape(NetworkOutput, (-1,1))
    hotencoder = OneHotEncoder(sparse=False)
    _ = hotencoder.fit(mapped_notes.reshape(-1,1))
    NetworkOutput = hotencoder.transform(NetworkOutput)
    
    # save the mapper and hotencoder to disk for prediction.
    #with open('./data/mapper.pkl','wb') as f:
    #    pickle.dump(mapper, f)
    #with open('./data/hotencoder.pkl','wb') as f:
    #    pickle.dump(hotencoder, f)
    
    print('Input shape = ',NetworkInput.shape, '\nOutput shape = ', NetworkOutput.shape)
    return mapped_notes, NetworkInput, NetworkOutput
#---------------------------------------
if debug : mapped_notes, NetworkInput, NetworkOutput = prepare_data(notes, n_vocab)

In [ ]:
#with open('./checkpoints/1/loss_stack.pkl','wb') as f:
#    pickle.dump(loss_stack, f)

In [37]:
import glob
import pickle

In [ ]:
#plt.plot(range(len(loss_stack)),loss_stack)
#plt.show()

In [38]:
music_files = glob.glob('./data/Mozart/*.pkl')
#assert len(music_files) > 0
#assert pieceId < len(music_files)
with open(music_files[5], 'rb') as f:
    data = pickle.load(f)

In [40]:
data[0][0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [ ]:
notes

In [ ]:
piano = parts.parts[0].recurse()

In [ ]:
names = set()
for p in piano :
    if isinstance(p, note.Note):
        names.add(p.name)
    if isinstance(p, chord.Chord):
        for pp in p.pitches:
            names.add(pp.name)

In [ ]:
lenoct = set()
octaves = set()
for index,file in enumerate([glob.glob('./music/Mozart/*.mid')[2]]):
        
    if index % 10 == 0 : print(index,end='')
    print('.',end='')
    midi = converter.parse(file)
    notes_to_parse  = []
    parts = instrument.partitionByInstrument(midi)

    if parts :
        piano_notes = parts.parts[0].recurse()
    else :
        print('\n*caution no parts are found.')
        piano_notes = parts.flat_notes

    # prepare notes
    notes_to_parse = []
    for elidx, element in enumerate(piano_notes):
            elinf = [ element_info.copy() for i in range(3)]
            if isinstance(element, note.Note):
                elinf[0][str(element.name)] = 1
                elinf[0][str(element.octave)] = 1
                elinf[1]['empty'] = 1
                elinf[2]['empty'] = 1
                notes_to_parse.append(elinf)
                octaves.add(element.octave)
                data[elidx,0] = elinfo[0].values()
            elif isinstance(element, chord.Chord):
                lenoct.add(len(element.pitches))
                assert len(element.pitches) <= 3
                for idx,e in enumerate(element.pitches) :
                    elinf[idx][str(e.name)] = 1
                    elinf[idx][str(e.octave)] = 1
                    octaves.add(e.octave)
                if idx != 2 :
                    for i in range(idx, 3):
                        elinf[i]['empty'] = 1
                notes_to_parse.append(elinf)
            else : # it shouldn't reach here.
                pass
            

In [ ]:
lenoct

In [ ]:
notes_to_parse

In [ ]:
note_names = ['A', 'B', 'B-', 'C', 'C#', 'D', 'E', 'E-', 'F', 'F#', 'G', 'G#']
octaves = ['1', '2', '3', '4', '5', '6']
element_info = OrderedDict({e:0 for e in ['empty'] + note_names + octaves})

In [ ]:
data = np.zeros([len(piano_notes), 3, 19], dtype=np.int32)

In [ ]:
data[0,0] = list(elinf[0].values())

In [ ]:
data[0]